In [ ]:

# 7장 시계열 데이터를 다뤄보자

### 작성 : [PinkWink](http://pinkwink.kr)

## 7-1. Numpy의 polyfit으로 회귀(regression) 분석하기
# pip install pandas_datareader
# conda install -c conda-forge fbprophet

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fbprophet import Prophet
from datetime import datetime

In [ ]:
# matpolib에 필요한 font 설정

path = "c:/Windows/Fonts/malgun.ttf"
import platform
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

plt.rcParams['axes.unicode_minus'] = False

In [ ]:
## Numpy의 polyfit으로 회귀(Regression) 분석하기

# csv 파일 정제

pinkwink_web = pd.read_csv('../data/08. PinkWink Web Traffic.csv',
                                          encoding='utf-8', thousands=',',
                                          names = ['date','hit'], index_col=0)
pinkwink_web = pinkwink_web[pinkwink_web['hit'].notnull()] #유입양이 없는 데이터 제외
pinkwink_web.head()

In [ ]:
# 유입량 그래프화
pinkwink_web['hit'].plot(figsize=(12,4), grid=True);

In [ ]:
# 선형 회귀 직선' 및 '다항 회귀식'을 표현

time = np.arange(0,len(pinkwink_web))
# len은 매개변수의 요소의 개수
# pinkwink_web index 값으로 이루어진 array 생성

traffic = pinkwink_web['hit'].values

fx = np.linspace(0, time[-1], 1000)
# linespace()함수는 두 수 사이를 50개의 균일한 간격의 수를 배열로 만들어준다.
# 주로 y축에 사용된다.

In [ ]:
# RMSE(Root Mean Square Error, 표준편차)을 계산해주는 error() 함수를 생성

def error(f, x, y):
    return np.sqrt(np.mean((f(x)-y)**2))

# f(x)는 예측값, y는 실제값
# sqrt-> 제곱근 함수. 루트 역할
# mean은 평균값 함수

In [ ]:
#  1차원 다항식
fp1 = np.polyfit(time, traffic, 1)
f1 = np.poly1d(fp1)

#  2차원 다항식
f2p = np.polyfit(time, traffic, 2)
f2 = np.poly1d(f2p)

# 3차원 다항식
f3p = np.polyfit(time, traffic, 3)
f3 = np.poly1d(f3p)

# 15차원 다항식
f15p = np.polyfit(time, traffic, 15)
f15 = np.poly1d(f15p)

# error 함수를 통해 오차의 표준편차 즉, 잔차 제곱합을 구한다.
# 데이터와 추정 모델 간의 불일치를 측정
print(error(f1, time, traffic))
print(error(f2, time, traffic))
print(error(f3, time, traffic))
print(error(f15, time, traffic))

In [ ]:
# 각 차원의 다항식을 그래프로 출력

plt.figure(figsize=(10,6))
plt.scatter(time, traffic, s=10)

plt.plot(fx, f1(fx), lw=4, label='f1')
plt.plot(fx, f2(fx), lw=4, label='f2')
plt.plot(fx, f3(fx), lw=4, label='f3')
plt.plot(fx, f15(fx), lw=4, label='f15')

plt.grid(True, linestyle='-', color='0.75')

plt.legend(loc=2)
plt.show()

## 7-2. Prophet 모듈을 이용한 forecast 예측

Prophet 모델의 주요 구성요소는 Trend, Seasonality, Holiday 입니다. 이 세가지를 결합하면 아래의 공식으로 나타낼 수 있습니다.
y(t)=g(t)+s(t)+h(t)+ϵi

- g(t): piecewise linear or logistic growth curve for modelling non-periodic changes in time series
- s(t): periodic changes (e.g. weekly/yearly seasonality)
- h(t): effects of holidays (user provided) with irregular schedules- ϵi: error term accounts for any unusual changes not accommodated by the model

위에서 Trend 를 g(t)함수는 주기적이지 않은 변화인 트렌드를 나타냅니다. 부분적으로 선형 또는 logistic 곡선으로 이루어져 있습니다. 그리고 Seasonality 인 s(t)함수는 weekly, yearly 등 주기적으로 나타나는 패턴들을 포함합니다.

Holiday를 나타내는 h(t)함수는 휴일과 같이 불규칙한 이벤트들을 나타냅니다. 만약 특정 기간에 값이 비정상적으로 증가 또는, 감소했다면, holiday로 정의하여 모델에 반영할 수 있습니다. 마지막으로 ϵi는 정규분포라고 가정한 오차입니다.


In [ ]:
df = pd.DataFrame({'ds':pinkwink_web.index, 'y':pinkwink_web['hit']})
df.reset_index(inplace=True)
df['ds'] =  pd.to_datetime(df['ds'], format="%y. %m. %d.")
del df['date']

m = Prophet(yearly_seasonality=True, daily_seasonality=True)
# 주기성이 연단위(yearly_seasonality) 및 일단위(daily_seasonality)로 있다고 알려준다
m.fit(df);

In [ ]:
# make_future_dataframe()로 지정된 날짜 수만큼 예측할 것
future = m.make_future_dataframe(periods=60)
future.tail()

In [1]:
forecast = m.predict(future)
forecast.head()

# 확인하고 싶은 변수들만 뽑아내서 확인
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

NameError: name 'm' is not defined

In [ ]:
# 시각화를 통해 예측 결과 확인
m.plot(forecast);

In [ ]:
# plot_components()를 사용해서 선형 회귀 및 계절성 성분 별로 분해
# 다시 말해, forecast component 시각화(Trend, Holidays, Weakly, Yearly, Daily)
m.plot_components(forecast);

## 7-3. Seasonal 시계열 분석으로 주식 데이터 분석하기

In [ ]:
from pandas_datareader import data
import yfinance as yf # Yahoo Finance에서 제공하는 데이터에 접근할 수 있는 라이브러리
yf.pdr_override()

start_date = '1990-1-1'
end_date = '2017-6-30'
KIA = data.get_data_yahoo('000270.KS', start_date, end_date)
KIA.head()

# KIA = web.DataReader('KRX:000270','google',start,end) # 구글용... 동작이 안됨
# KIA = web.DataReader('000270.KS','yahoo',start,end) # 구글용... 동작이 안됨


In [ ]:
# 종가를 기준으로 시각화
KIA['Close'].plot(figsize=(12,6), grid=True);

In [ ]:
# 일부 데이터를 잘라서 먼저 forecast
KIA_trunc = KIA[:'2016-12-31']
KIA_trunc

In [ ]:
df = pd.DataFrame({'ds':KIA_trunc.index, 'y':KIA_trunc['Close']})
df.reset_index(inplace=True)
del df['Date']
df.head()

In [ ]:
# 주기성이 '일단위(daily_seasonality)'로 있다고 알려준다
#  prophet object를 생성하고, 훈련 데이터를 fitting 하여  prophet 모델을 만듭니다.
m = Prophet(daily_seasonality=True)
m.fit(df);

In [ ]:
# 1년(365일) 후. 즉, 2017년 12월 31일까지의 데이터를 예측하겠다는 의미
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
m.plot(forecast);

In [ ]:
# 계절성 성분 별로 분해해서 분석.
m.plot_components(forecast);

In [ ]:
# 받아오는 데이터 날짜 변경
start_date = '2014-1-1'
end_date = '2017-7-31'
KIA = data.get_data_yahoo('000270.KS', start_date, end_date)
KIA['Close'].plot(figsize=(12,6), grid=True);

In [ ]:
# 예측용으로 사용할 데이터
KIA_trunc = KIA[:'2017-05-31']
KIA_trunc['Close'].plot(figsize=(12,6), grid=True);

In [ ]:
df = pd.DataFrame({'ds':KIA_trunc.index, 'y':KIA_trunc['Close']})
df.reset_index(inplace=True)
del df['Date']

In [ ]:
# 주기성 일단위
m = Prophet(daily_seasonality=True)
m.fit(df);

In [ ]:
future = m.make_future_dataframe(periods=61)
future.tail()

In [ ]:
forecast = m.predict(future)
m.plot(forecast);

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(KIA.index, KIA['Close'], label='real')
plt.plot(forecast['ds'], forecast['yhat'], label='forecast')
plt.grid()
plt.legend()
plt.show()

## 7-4 Growth Model, 예제에서 사용한 데이터는 주기성을 띠면서, 점점 성장하는(?) 모습의 데이터

In [ ]:
df = pd.read_csv('../data/08. example_wp_R.csv')
df.head()


In [ ]:
df['y'] = np.log(df['y']) # 로그 변환
df['cap'] = 8.5 # 예측 값의 상한을 8.5로 설정
df.head()

In [ ]:
m = Prophet(growth='logistic', daily_seasonality=True)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=1826)
future['cap'] = 8.5
fcst = m.predict(future)
m.plot(fcst);

In [ ]:
forecast = m.predict(future)
m.plot_components(forecast);

## holiday forecast
휴일이나 모델에 반영하고 싶은 이벤트가 있으면, Dataframe을 생성해 반영할 수 있다
Prophet 객체를 생성할 때, holidays 파라미터를 사용하면 된다

In [ ]:
df = pd.read_csv('../data/08. example_wp_peyton_manning.csv')
df['y'] = np.log(df['y']) # 로그 변환
m = Prophet(daily_seasonality=True)
m.fit(df)
future = m.make_future_dataframe(periods=366)

In [ ]:
df.y.plot(figsize=(12,6), grid=True);

In [ ]:
# 예제용 데이터 프레임 생성
# playoffs DataFrame +  superbowls DataFrame을 합쳐서(concat),
# holidays라는 DataFrame을 생성
# playoffs DataFrame는 4개의 변수를 갖는다.

playoffs = pd.DataFrame({
  'holiday': 'playoff',
  'ds': pd.to_datetime(['2008-01-13', '2009-01-03', '2010-01-16',
                        '2010-01-24', '2010-02-07', '2011-01-08',
                        '2013-01-12', '2014-01-12', '2014-01-19',
                        '2014-02-02', '2015-01-11', '2016-01-17',
                        '2016-01-24', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
superbowls = pd.DataFrame({
  'holiday': 'superbowl',
  'ds': pd.to_datetime(['2010-02-07', '2014-02-02', '2016-02-07']),
  'lower_window': 0,
  'upper_window': 1,
})
holidays = pd.concat((playoffs, superbowls))
holidays.head()

In [ ]:
# Prophet 객체를 생성할 때, holidays 파라미터를 사용

m = Prophet(holidays=holidays, daily_seasonality=True)
forecast = m.fit(df).predict(future)

In [ ]:
forecast[(forecast['playoff'] + forecast['superbowl']).abs() > 0][
        ['ds', 'playoff', 'superbowl']][-10:]

In [ ]:
m.plot(forecast);

In [ ]:
m.plot_components(forecast);